In [1]:
import boto3
import yaml
from src.logger import setup_logger
from src.minio_uploader import upload_to_minio

# Set up logger
pipeline_name = "pangenome"
target_table = "genome"
schema = "pangenome"

logger = setup_logger(
    pipeline_name=pipeline_name,
    target_table=target_table,
    schema=schema
)

logger.info("Notebook started")

from config.config_loader import ConfigLoader

# Path to config file in MinIO
config_path = "genome"

# Load config
loader = ConfigLoader(config_path, logger)
loader.load_and_validate()

# (Optional) Print something
print(loader.get_target_table())

# Start Spark
from src.spark_session import start_spark_session
spark = start_spark_session(logger=logger)

{"time": "2025-07-08 16:33:31,932", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "2161914337", "msg": "Notebook started"}
{"time": "2025-07-08 16:33:31,944", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "config_loader", "msg": "ConfigLoader initialized for target table: genome"}
{"time": "2025-07-08 16:33:31,945", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "config_loader", "msg": "Resolved config path: s3a://cdm-lake/config-json/genome.json"}
{"time": "2025-07-08 16:33:31,946", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "config_loader", "msg": "Loading config from MinIO: bucket=cdm-lake, key=config-json/genome.json"}
{"time": "2025-07-08 16:33:32,100", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "config_loader", "msg": "Config loaded succes

In [ ]:

# Validate input files
from src.input_file_validator import validate_input_files
validate_input_files(loader, logger)


# Check schema headers
from src.validate_schema import validate_schema_against_file
validate_schema_against_file(loader, logger)


# Check TSV format
#from src.input_file_validator import validate_input_files
#validate_input_files(loader)

# Run validations
from src.run_validations import run_validations_from_config
run_validations_from_config(loader, logger)


# Referential integrity
from src.run_referential_integrity_checks import run_referential_integrity_checks
run_referential_integrity_checks(spark, loader, logger)


#import importlib
#import src.run_great_expectations_validations

#importlib.reload(src.run_great_expectations_validations)




In [2]:
# Great Expectations
from src.run_great_expectations_validations import run_great_expectations_validation
run_great_expectations_validation(spark, loader, logger)


# Upload log file to MinIO
upload_to_minio(logger.log_file_path)

{"time": "2025-07-08 16:33:49,139", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "DEBUG", "module": "config_loader", "msg": "Target table: pangenome.genome"}
{"time": "2025-07-08 16:33:49,140", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Starting Great Expectations validation for table: pangenome.genome with suite: default_suite"}
{"time": "2025-07-08 16:33:54,405", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Loaded Spark table: pangenome.genome"}
{"time": "2025-07-08 16:33:54,559", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Great Expectations context initialized."}
{"time": "2025-07-08 16:33:54,578", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "lev

Calculating Metrics: 0it [00:00, ?it/s]

{"time": "2025-07-08 16:33:57,812", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Validation run completed."}
{"time": "2025-07-08 16:33:57,814", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "DEBUG", "module": "run_great_expectations_validations", "msg": "Validation summary: {'success': True, 'successful_expectations': 0, 'unsuccessful_expectations': 0, 'success_percent': None}"}
{"time": "2025-07-08 16:33:58,804", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Data Docs built after validation."}


Calculating Metrics: 0it [00:00, ?it/s]

{"time": "2025-07-08 16:33:59,805", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Checkpoint executed."}
{"time": "2025-07-08 16:34:00,963", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Data Docs rebuilt with checkpoint results."}
✅ GE validation and checkpoint complete. Data Docs generated.
✅ Uploaded log to MinIO at: s3://cdm-lake/logs/pangenome/pipeline_run_20250708_163331.log
